- Implement a multi-agent Retrieval-Augmented Generation (RAG) system.  
- Make LLM API calls via Together.AI.  
- Create user interfaces for AI applications using Gradio.  
- We'll integrate Gradio’s interactive UI components with large language models to develop a multi-agent RAG system that enhances information retrieval and generation.  
- Each section includes a Module to test different LLM-based Capabilities



In [ ]:
# import libraries

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
from io import BytesIO
import requests
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
try:
    import gradio as gr
    from together import Together
except ImportError:
    !pip install -q together
    !pip install -q gradio
import gradio as gr
from together import Together
import numpy as np

In [ ]:

# Get Client
your_api_key = "x"

client = Together(api_key=your_api_key)

def prompt_llm(prompt, show_cost=False):
    # This function allows us to prompt an LLM via the Together API

    # model
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"

    # Calculate the number of tokens
    tokens = len(prompt.split())

    # Calculate and print estimated cost for each model
    if show_cost:
        print(f"\nNumber of tokens: {tokens}")
        cost = (0.1 / 1_000_000) * tokens
        print(f"Estimated cost for {model}: ${cost:.10f}\n")

    # Make the API call
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

def gen_image(prompt, width=256, height=256):
    # This function allows us to generate images from a prompt
    response = client.images.generate(
        prompt=prompt,
        model="stabilityai/stable-diffusion-xl-base-1.0",  # Using a supported model
        steps=30,
        n=1,
    )
    image_url = response.data[0].url
    image_filename = "image.png"

    # Download the image using requests instead of wget
    response = requests.get(image_url)
    with open(image_filename, "wb") as f:
        f.write(response.content)
    img = Image.open(image_filename)
    img = img.resize((height, width))

    return img

print("Lets GOOO")


In [ ]:
class EmailResponseRetriever:
    def __init__(self):
        self.encoder = SentenceTransformer("all-MiniLM-L6-v2")
        self.examples = {
            "event_promotion": """
                ORIGINAL POST:
                We have 15 more spots for our AI Event tomorrow. Need a catchy announcement.

                MY RESPONSE:
                🚨 Only 15 spots left for our AI bash tomorrow! 🚀 
                Get ready for mind-blowing insights on AI trends, a hands-on workshop for non-coders, 
                and of course, 🍕 delicious pizza! 
                Let's finish February strong—RSVP now: LINK
                #VancouverAI #TechEvents #PizzaAndAI
            """,
            "speaker_highlight": """
                ORIGINAL POST:
                Promote a talk by George Chen on AI in digital marketing.

                MY RESPONSE:
                🚀 You're Invited! 
                Join us for an inspiring talk by George Chen, digital marketing expert with over 15 years of experience, 
                including as Director of Digital Marketing at Barneys New York. 
                Discover how AI is transforming analytics, content creation, and ad targeting. 
                🍕 Plus, enjoy some pizza and networking! 
                RSVP here: LINK
                #AIinMarketing #VancouverTech #LearnWithUs
            """,
            "event_recap": """
                ORIGINAL POST:
                Share highlights from last Friday's in-person session.

                MY RESPONSE:
                🎉 What an incredible AI session last Friday! 
                From exploring AutoML and Robotics 2.0 to AI in mental health, 
                our host Issam Laradji and guest speaker George Chen delivered invaluable insights. 
                We wrapped up with pizza, bubbles, and amazing networking. 🍕🥂 
                Don't miss our next session: LINK
                #VancouverAI #CommunityVibes #TechLearning
            """,
            "community_engagement": """
                ORIGINAL POST:
                Ask a fun question to boost community interaction.

                MY RESPONSE:
                🤖 If you could automate one annoying task with AI, what would it be? 
                Folding laundry? Scheduling meetings? Making small talk with your in-laws? 😂 
                Drop your ideas below! #AI #Automation #TechTalk
            """,
            "funny_trend_hijacking": """
                ORIGINAL POST:
                That new 'distracted boyfriend' meme is huge! How can we use it?

                MY RESPONSE:
                (Image: Distracted Boyfriend meme) 
                Boyfriend: Me. 
                Distracted: Our new AI tool. 
                Girlfriend: My old, clunky workflow. 
                Sometimes, you’ve just got to upgrade! 😉 
                #MemeMarketing #AIHumor #StayFocused
            """,
            "educational_content": """
                ORIGINAL POST:
                Explain what Python is in a fun and simple way.

                MY RESPONSE:
                🐍 Python: Not just a snake, but the coding language that makes your favorite apps run smooth as butter. 
                Whether it's automating tasks or building chatbots, Python’s got your back! 
                Want to learn more? Join our next session: LINK
                #LearnToCode #PythonRocks #TechForEveryone
            """,
            "ethical_ai_awareness": """
                ORIGINAL POST:
                Share a post about XX YY's research on ethical AI.

                MY RESPONSE:
                🌐 Big thanks to XX YY for her incredible talk on the ethical use of AI! 
                She highlighted the importance of addressing biases and promoting responsible implementations. 
                Missed it? No worries! Join us at our next event: LINK
                #EthicalAI #VancouverAI #ResponsibleTech
            """,
            "interactive_post": """
                ORIGINAL POST:
                Engage the community with an interactive challenge.

                MY RESPONSE:
                💡 Challenge Alert! 
                Build an interactive text-based game using Python—no experience needed! 
                Our host AA will guide you step-by-step. 
                Think you’ve got what it takes? Join us and let’s play! 
                RSVP here: LINK
                #PythonChallenge #LearnByDoing #VancouverAI
            """,
            "bilingual_twist": """
                ORIGINAL POST:
                Add a Portuguese twist to a community post.

                MY RESPONSE:
                🎉 AI + Pizza? Só vem! 🍕🤖 
                Join us for our next AI session—learn, connect, and enjoy! 
                "Só vem" means "Just come" in Portuguese, and we promise it’ll be worth it! 
                RSVP: LINK
                #VancouverAI #TechCommunity #Aprendizado
            """,
        }
        self.example_embeddings = {
            k: self.encoder.encode(v) for k, v in self.examples.items()
        }

    def get_relevant_response(self, query, top_k=2):
        query_embedding = self.encoder.encode(query)
        similarities = {
            k: cosine_similarity([query_embedding], [emb])[0][0]
            for k, emb in self.example_embeddings.items()
        }
        sorted_examples = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

        relevant_examples = []
        for example_name, score in sorted_examples[:top_k]:
            if score > 0.5:
                relevant_examples.append(self.examples[example_name])
        return "\n".join(relevant_examples) if relevant_examples else "No relevant examples found."

class EmailAgent:
    def __init__(self, role, client):
        self.role = role
        self.client = client
        self.response_retriever = EmailResponseRetriever()

        self.prompts = {
            "analyzer": """SYSTEM: You are an expert social media post analyzer for a tech-focused AI community.
        Your role is to break down social media posts into key components and provide clear, actionable insights.

        INSTRUCTIONS:
        • Extract main topics and key points from the original post
        • Determine engagement potential (Low, Medium, High)
        • List all required actions in bullet points
        • Analyze the tone of the message (formal, informal, humorous, educational, etc.)
        • Consider similar past responses
        • Highlight any compliance or brand consistency concerns
        • Limit response to 50 words maximum
        • Show response only without additional commentary

        SIMILAR PAST RESPONSES:
        {examples}

        ORIGINAL POST: {content}""",
            "drafter": """SYSTEM: You are a professional social media content creator for an AI and tech community.
        Draft posts that align with our past successful social media content while maintaining brand voice and engagement focus.

        INSTRUCTIONS:
        • Address all key points from the original post
        • Use a friendly, creative, and tech-savvy tone like in our examples
        • Ensure all content aligns with brand guidelines and compliance standards
        • Include clear calls to action and engagement prompts
        • Maintain a professional yet approachable tone
        • Add necessary disclaimers where applicable
        • Limit response to 50 words maximum
        • Show response only without additional commentary

        SIMILAR PAST RESPONSES:
        {examples}

        Based on this analysis: {content}""",
            "reviewer": """SYSTEM: You are a senior social media quality assurance specialist for a tech and AI community.
        Ensure social media posts meet brand communication standards and match our engaging and friendly tone.

        INSTRUCTIONS:
        • Verify alignment with example responses
        • Check for brand compliance and appropriateness
        • Assess professional yet friendly and engaging tone
        • Review completeness of the response
        • Evaluate the handling of sensitive or technical information
        • Confirm all action items and engagement prompts are clearly stated
        • Limit response to 50 words maximum
        • Show response only without additional commentary

        SIMILAR PAST RESPONSES:
        {examples}

        Evaluate this draft post: {content}""",
            "sentiment": """SYSTEM: You are an expert in analyzing sentiment and emotional context in social media communications
        within the AI and tech community.

        INSTRUCTIONS:
        • Analyze overall sentiment (positive, negative, neutral)
        • Identify emotional undertones (humorous, motivational, educational)
        Detect urgency or engagement triggers
        • Assess community or audience satisfaction level
        • Flag any concerning or off-brand language
        • Recommend tone adjustments if needed
        • Limit response to 50 words maximum
        • Show response only without additional commentary

        ORIGINAL POST: {content}""",
            "example_justifier": """SYSTEM: You are an example matching expert. In 2 lines, explain why the following example responses are relevant
        to this social media content. Be specific and concise.

        ORIGINAL POST: {content}
        SIMILAR PAST RESPONSES: {examples}""",
        }

        sample_posts = [
        """Hey! I want to create a new post talking about how AI is transforming education, especially in universities.
        Can we highlight some cool use cases and make it engaging for students and educators alike?""",
        
        """Hello! I'd like to draft a post about today's event. 
        Tonight, we covered topics like Autonomous AI, AI in Mental Health, and how DeepMind is improving energy efficiency.
        Let's make it a fun and insightful recap for our community!""",
        
        """We just finished an amazing in-person session! 
        Our guest speaker talked about the ethical use of AI and we even had a live demo of building an interactive text-based game using Python.
        I’d love to share a post to recap the event and invite folks to our next meetup!""",
        
        """I'm thinking of a community engagement post. 
        Maybe ask a question like: 'If you could automate one part of your daily routine with AI, what would it be?' 
        I want it to spark fun and creative responses!""",
        
        """Hi there! We're hosting a webinar next week about using AI in digital marketing. 
        Can we create a catchy post to drive sign-ups? 
        Something that highlights our expert speaker and the practical insights attendees will gain.""",
        
        """Hey! I’d like to share a quick highlight of our last online session. 
        We discussed AI applications in law, the food industry, marketing, and electrical engineering.
        Let's make it engaging and showcase the diverse perspectives shared!""",
        
        """I'd love to share a post thanking our volunteer team for their hard work during last week's event. 
        It’d be great to include their names and maybe a fun emoji or two to show our appreciation!""",
        
        """We’ve got only 10 spots left for our AI workshop tomorrow! 
        Can we create an urgent, FOMO-inducing post to encourage last-minute RSVPs? 
        Let's make it fun but clear on how valuable the session will be.""",
        
        """I'm thinking about creating a meme for our social media page. 
        Maybe using the 'distracted boyfriend' meme but with an AI twist. 
        Something that tech folks would find funny and shareable!""",
        
        """I’d like to make a post celebrating our latest project milestone. 
        We've successfully implemented an AI-driven feature in our app, and I want to show the community how this improves their experience!"""
    ]

    def process(self, content):
        # Get relevant example responses for the email content
        relevant_examples = self.response_retriever.get_relevant_response(content)
        sample_examples = "\n".join(sample_posts)
        combined_examples = f"{relevant_examples}\n{sample_examples}"

        return self.prompt_llm(
            self.prompts[self.role].format(content=content, examples=combined_examples)
)
        
    def prompt_llm(self, prompt):
        try:
            response = self.client.chat.completions.create(
                model="meta-llama/Meta-Llama-3-8B-Instruct-Lite",
                messages=[{"role": "user", "content": prompt}],
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error: {e}"

def gen_image(prompt, width=256, height=256):
    try:
        response = client.images.generate(
            prompt=prompt,
            model="stabilityai/stable-diffusion-xl-base-1.0",
            steps=30,
            n=1,
        )
        image_url = response.data[0].url
        image_data = requests.get(image_url).content
        img = Image.open(BytesIO(image_data))
        img = img.resize((width, height))
        return img
    except Exception as e:
        return f"Error generating image: {e}"

def create_gradio_interface():
    def process_all_agents(user_input):
        agents = {
            "analyzer": EmailAgent(role="analyzer", client=client),
            "drafter": EmailAgent(role="drafter", client=client),
            "reviewer": EmailAgent(role="reviewer", client=client),
            "sentiment": EmailAgent(role="sentiment", client=client),
            "example_justifier": EmailAgent(role="example_justifier", client=client),
        }

        results = {
            role: agent.process(user_input) for role, agent in agents.items()
        }

        # Generate an image based on the combined results or a key result like the drafter's output
        combined_result = "\n".join(results.values())
        image = gen_image(combined_result)

        return (
            results["analyzer"], results["drafter"], results["reviewer"],
            results["sentiment"], results["example_justifier"],
            image
        )

    with gr.Blocks(title="Social Media Helper") as interface:
        gr.Markdown("# 📲 Social Media Helper")
        gr.Markdown("### Get insights, create posts, and ensure brand compliance for social media!")

        email_input = gr.Textbox(
            label="📝 Enter Your Social Media Post Content",
            placeholder="Type your post content here....",
            lines=4,
            scale=2,
            container=True,
        )

        process_all_btn = gr.Button("🔄 Generate Social Media Insights", variant="primary")

        with gr.Row():
            with gr.Column():
                analyzer_output = gr.Textbox(label=" 📊 Key Insights", lines=4, container=True)
                drafter_output = gr.Textbox(label="👀 Content Preview", lines=4, container=True)
                reviewer_output = gr.Textbox(label="😊 Reviewer Output", lines=4, container=True)

            with gr.Column():
                sentiment_output = gr.Textbox(label="👍 Sentiment Analysis", lines=4, container=True)
                example_justifier_output = gr.Textbox(label="📝 Why?", lines=4, container=True)
                image_output = gr.Image(label="🖼️ Generated Image", type="pil")

        process_all_btn.click(
            fn=process_all_agents,
            inputs=[email_input],
            outputs=[
                analyzer_output, drafter_output, reviewer_output,
                sentiment_output, example_justifier_output,
                image_output
            ],
        )

    return interface

if __name__ == "__main__":
    interface = create_gradio_interface()
    interface.launch()